# SONATA 용 IPL 엑셀 파일을 APECS 위협 라이브러리로 변환
* 버젼 : 1.0
* 날짜 : 2022-11-18

In [1]:
import xlwings as xw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcl
import seaborn as sns

from datetime import datetime
import sqlite3

import random

# Machine learning 
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA


pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'


# 엑셀용 IPL 데이터 불러오기

In [2]:
raw_xl = xw.Book( 'TestLibrary_시험용(헤더 수정).xlsx')

sh = raw_xl.sheets(1)

# 위협 라이브로리 로딩
n_df_1 = sh.range('A1').options(pd.DataFrame, expand='table').value
n_df_1.insert( 4, 'Delta Freq(MHz)', n_df_1['FreqHigh(MHz)'] - n_df_1['FreqLow(MHz)'] )
n_df_1.insert( 4, 'CenterFreq(MHz)', ( n_df_1['FreqHigh(MHz)'] + n_df_1['FreqLow(MHz)'] ) / 2.0 )

n_df_1.insert( 13, 'Delta PRI(usec)', n_df_1['PRIHigh(usec)'] - n_df_1['PRILow(usec)'] )
n_df_1.insert( 13, 'CenterPRI(usec)', ( n_df_1['PRIHigh(usec)'] + n_df_1['PRILow(usec)'] ) / 2.0 )

n_df_1.insert( 24, 'Delta PW(usec)', n_df_1['PW High(usec)'] - n_df_1['PW Low(usec)'] )
n_df_1.insert( 24, 'CenterPW(usec)', ( n_df_1['PW High(usec)'] + n_df_1['PW Low(usec)'] ) / 2.0 )

#n_df_1.index = n_df_1.index.map( lambda x : int(x) )

In [3]:
n_df_1


,Pulse Type,RF Type,FreqLow(MHz),FreqHigh(MHz),CenterFreq(MHz),Delta Freq(MHz),SwitchLevel,PatternType,PatternPeriodLow(usec),PatternPeriodHigh(usec),PRI Type,PRILow(usec),PRIHigh(usec),CenterPRI(usec),Delta PRI(usec),PRF Low(usec),PRF High(usec),Jitter(%),Stagger/Switch Level,PRI Pattern Type,PRI PatternPeriodLow(usec),PRI PatternPeriodHigh(usec),PW Low(usec),PW High(usec),CenterPW(usec),Delta PW(usec),AS Type,AS Low(usec),AS High(usec),Threat\nLevel,DeleteTime(sec),EmitterNotation,RadarMode Name,Category,First Tech No,Second Tech No,Jamming Threshold,Dwell Freq1 Low(MHz),Dwell Freq1 High(MHz),Dwell Freq2 Low(MHz),Dwell Freq2 High(MHz),Dwell Freq3 Low(MHz),Dwell Freq3 High(MHz),Dwell Freq4 Low(MHz),Dwell Freq4 High(MHz),Dwell Freq5 Low(MHz),Dwell Freq5 High(MHz),Dwell Freq6 Low(MHz),Dwell Freq6 High(MHz),Dwell Freq7 Low(MHz),Dwell Freq7 High(MHz),Dwell Freq8 Low(MHz),Dwell Freq8 High(MHz),Dwell Freq9 Low(MHz),Dwell Freq9 High(MHz),Dwell Freq10 Low(MHz),Dwell Freq10 High(MHz),Dwell Freq11 Low(MHz),Dwell Freq11 High(MHz),Dwell Freq12 Low(MHz),Dwell Freq12 High(MHz),Dwell Freq13 Low(MHz),Dwell Freq13 High(MHz),Dwell Freq14 Low(MHz),Dwell Freq14 High(MHz),Dwell Freq15 Low(MHz),Dwell Freq15 High(MHz),Dwell Freq16 Low(MHz),Dwell Freq16 High(MHz),SD Value1 Low(MHz),SD Value1 High(MHz),SD Value2 Low(MHz),SD Value2 High(MHz),SD Value3 Low(MHz),SD Value3 High(MHz),SD Value4 Low(MHz),SD Value4 High(MHz),SD Value5 Low(MHz),SD Value5 High(MHz),SD Value6 Low(MHz),SD Value6 High(MHz),SD Value7 Low(MHz),SD Value7 High(MHz),SD Value8 Low(MHz),SD Value8 High(MHz),SD Value9 Low(MHz),SD Value9 High(MHz),SD Value10 Low(MHz),SD Value10 High(MHz),SD Value11 Low(MHz),SD Value11 High(MHz),SD Value12 Low(MHz),SD Value12 High(MHz),SD Value13 Low(MHz),SD Value13 High(MHz),SD Value14 Low(MHz),SD Value14 High(MHz),SD Value15 Low(MHz),SD Value15 High(MHz),SD Value16 Low(MHz),SD Value16 High(MHz),Auto\nWarning,ERP(W),Platform 1 No,Platform 1 Name,Platform 2 No,Platform 2 Name,Platform 3 No,Platform 3 Name,Platform 4 No,Platform 4 Name,Platform 5 No,Platform 5 Name
No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38.0,1.0,1.0,9240.0,9490.0,9365.0,250.0,0.0,0.0,0.0,0.0,1.0,2257.0,2278.0,2267.5,21.0,439.0,443.1,0.0,0.0,0.0,0.0,0.0,0.8,1.3,1.05,0.5,1.0,2600000.0,3200000.0,8.0,0.0,A304C,MUSHROOM 1,1.0,533.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,C/RU B-5,2.0,TU-95,3.0,-,4.0,-,5.0,-
1.0,1.0,2.0,7800.0,8210.0,8005.0,410.0,0.0,0.0,0.0,0.0,3.0,587.0,593.0,590.0,6.0,1686.3,1703.6,5.0,0.0,0.0,0.0,0.0,0.4,0.8,0.60,0.4,5.0,0.0,0.0,9.0,0.0,A374A,No Name,1.0,1761.0,531.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,STYX,2.0,-,3.0,-,4.0,-,5.0,-
2.0,1.0,1.0,7800.0,8210.0,8005.0,410.0,0.0,0.0,0.0,0.0,3.0,587.0,593.0,590.0,6.0,1686.3,1703.6,5.0,0.0,0.0,0.0,0.0,0.4,0.8,0.60,0.4,5.0,0.0,0.0,9.0,0.0,A374A,No Name,1.0,1761.0,531.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,STYX,2.0,-,3.0,-,4.0,-,5.0,-
3.0,1.0,2.0,7927.0,8210.0,8068.5,283.0,0.0,0.0,0.0,0.0,3.0,509.0,638.0,573.5,129.0,1567.4,1964.6,5.0,0.0,0.0,0.0,0.0,0.4,0.8,0.60,0.4,5.0,0.0,0.0,9.0,0.0,A374A,No Name,1.0,1761.0,531.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [4]:
n_df_1.describe()

,Pulse Type,RF Type,FreqLow(MHz),FreqHigh(MHz),CenterFreq(MHz),Delta Freq(MHz),SwitchLevel,PatternType,PatternPeriodLow(usec),PatternPeriodHigh(usec),PRI Type,PRILow(usec),PRIHigh(usec),CenterPRI(usec),Delta PRI(usec),PRF Low(usec),PRF High(usec),Jitter(%),Stagger/Switch Level,PRI Pattern Type,PRI PatternPeriodLow(usec),PRI PatternPeriodHigh(usec),PW Low(usec),PW High(usec),CenterPW(usec),Delta PW(usec),AS Type,AS Low(usec),AS High(usec),Threat\nLevel,DeleteTime(sec),Category,First Tech No,Second Tech No,Jamming Threshold,Dwell Freq1 Low(MHz),Dwell Freq1 High(MHz),Dwell Freq2 Low(MHz),Dwell Freq2 High(MHz),Dwell Freq3 Low(MHz),Dwell Freq3 High(MHz),Dwell Freq4 Low(MHz),Dwell Freq4 High(MHz),Dwell Freq5 Low(MHz),Dwell Freq5 High(MHz),Dwell Freq6 Low(MHz),Dwell Freq6 High(MHz),Dwell Freq7 Low(MHz),Dwell Freq7 High(MHz),Dwell Freq8 Low(MHz),Dwell Freq8 High(MHz),Dwell Freq9 Low(MHz),Dwell Freq9 High(MHz),Dwell Freq10 Low(MHz),Dwell Freq10 High(MHz),Dwell Freq11 Low(MHz),Dwell Freq11 High(MHz),Dwell Freq12 Low(MHz),Dwell Freq12 High(MHz),Dwell Freq13 Low(MHz),Dwell Freq13 High(MHz),Dwell Freq14 Low(MHz),Dwell Freq14 High(MHz),Dwell Freq15 Low(MHz),Dwell Freq15 High(MHz),Dwell Freq16 Low(MHz),Dwell Freq16 High(MHz),SD Value1 Low(MHz),SD Value1 High(MHz),SD Value2 Low(MHz),SD Value2 High(MHz),SD Value3 Low(MHz),SD Value3 High(MHz),SD Value4 Low(MHz),SD Value4 High(MHz),SD Value5 Low(MHz),SD Value5 High(MHz),SD Value6 Low(MHz),SD Value6 High(MHz),SD Value7 Low(MHz),SD Value7 High(MHz),SD Value8 Low(MHz),SD Value8 High(MHz),SD Value9 Low(MHz),SD Value9 High(MHz),SD Value10 Low(MHz),SD Value10 High(MHz),SD Value11 Low(MHz),SD Value11 High(MHz),SD Value12 Low(MHz),SD Value12 High(MHz),SD Value13 Low(MHz),SD Value13 High(MHz),SD Value14 Low(MHz),SD Value14 High(MHz),SD Value15 Low(MHz),SD Value15 High(MHz),SD Value16 Low(MHz),SD Value16 High(MHz),Auto\nWarning,ERP(W),Platform 1 No,Platform 2 No,Platform 3 No,Platform 4 No,Platform 5 No
count,2000.0,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.0,2000.0,2000.0,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.0,2000.0,2000.0,2000.000000,2000.0000,2000.000000,2000.00000,2000.000000,2.000000e+03,2.000000e+03,2000.000000,2000.0,2000.000000,2000.000000,2000.000000,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.00000,2000.000000,2000.00000,2000.00000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
mean,1.0,1.20400,7915.781100,8236.329000,8076.055050,320.547900,0.0,0.0,0.0,0.0,1.899500,1028.891250,1272.837850,1150.864550,243.946600,2592.063450,3619.657150,3.33700,0.384000,0.0,0.0,0.0,3.042700,9.7703,6.406500,6.72760,1.800500,2.861765e+06,5.655994e+06,6.018500,0.0,2.112000,82.019500,43.540000,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.87475,52.850000,41.01390,55.99065,32.06565,47.036250,28.028100,42.402650,14.151750,20.409650,12.405350,18.656550,7.909450,13.615050,7.917450,13.623050,7.149450,12.85505,7.149450,12.85505,7.149450,12.85505,7.149450,12.85505,7.149450,12.85505,7.149450,12.85505,7.149450,12.85505,7.149450,12.85505,0.280500,1.0,1.0,2.0,3.0,4.0,5.0
std,0.0,0.40307,3338.585028,3417.908059,3372.777501,392.401176,0.0,0.0,0.0,0.0,1.019765,1009.110911,1351.641749,1099.525829,924.474067,6516.574832,8628.319215,5.15031,1.951781,0.0,0.0,0.0,7.663031,23.9772,14.192576,21.48347,1.484521,3.099485e+06,5.879270e

# SQLITE 연동

In [5]:
# DB 생성
#conn = sqlite3.connect( "cedeob.sqlite3", isolation_level=None )
conn = sqlite3.connect( "cedeob.sqlite3" )

# 커서 연결
cur = conn.cursor()

# 레이더/레이더모드 관련 테이블 삭제

In [6]:
now = datetime.now()
nowStr = now.strftime( '%Y-%m-%d %H:%M:%S' )

print( f"현재 시간은 {nowStr} 입니다.\n\n" )

try:
    cur.execute( 'delete from RADAR_RF_SEQUENCE_NAME;')
    cur.execute( 'delete from RADAR_RF_SEQUENCE;')
    cur.execute( 'delete from RADAR_RF_VALUES;')
    cur.execute( 'delete from RADAR_PRI_SEQUENCE_NAME;')
    cur.execute( 'delete from RADAR_PRI_SEQUENCE;')
    cur.execute( 'delete from RADAR_PRI_VALUES;')

    cur.execute( 'delete from RADARMODE;')
                  
    cur.execute( 'delete from RADAR_COMMENTS;')

    cur.execute( 'delete from RADAR_RF_SPOT_VALUES;')
    cur.execute( 'delete from RADAR_PRI_SPOT_VALUES;')

    cur.execute( 'delete from RADARMODE_LIFECYCLE;')

    cur.execute( 'delete from ASSOC_PLATFORM;')
    cur.execute( 'delete from PLATFORM;')

    cur.execute( 'delete from RADAR;')
    
    print( "===> 테이블을 정상적으로 삭제 했습니다." )
    
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_COMMENTS';")
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_RF_VALUES';")
    
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_RF_SEQUENCE';")
    
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_PRI_VALUES';")
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_PRI_SEQUENCE';")
    
    print( "===> 자동 증가를 초기화 했습니다." )

    conn.commit()
    
except Exception as exception:
    print( exception )
    print( '에러 발생함' )    


현재 시간은 2023-01-06 11:01:52 입니다.


database is locked
에러 발생함


# 레이더 테이블 생성

In [8]:
elnot_df = n_df_1['EmitterNotation'].unique()
elnot_df = np.sort( elnot_df )

platform_index = 0

try:
    for index, value in enumerate(elnot_df):
        # 레이더 정보 저장
        sql = f"INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DATE_LAST_REVIEWED, DATE_ACTIVATED, DATE_INACTIVATED, TIME_INACTIVATED, COMBINED_DATE_LAST_UPDATED ) VALUES ( {index+1}, '{value}', '레이더_{index+1}', 100, 'ACTIVE', 1, 1, '', '', '', '', 200, '' )"
        #print( sql );
        cur.execute( sql )
        
        # 레이더 커멘트 정보 저장
        sql = f"INSERT INTO RADAR_COMMENTS (RADAR_INDEX, TITLE, DATE_CREATED, DATE_LAST_UPDATED, COMMENTS) VALUES ( 1, '레이더 커멘트#{index+1}', '', '', '커멘트를 작성하면 됩니다.')"
        #print( sql )
        cur.execute( sql )
        
        # 레이더 플레폼 저장
        rows = n_df_1[ n_df_1['EmitterNotation'] == value ]
        platform_name = rows.iloc[0][ ['Platform 1 Name', 'Platform 2 Name', 'Platform 3 Name', 'Platform 4 Name', 'Platform 5 Name'] ]
        #print( type(platform_name[0]) )
        
        for i in range(5):
            if platform_name[i] != '-' and platform_name[i] != '':
                platform_index = platform_index + 1
                
                sql = f"INSERT INTO ASSOC_PLATFORM(RADAR_INDEX, PLATFORM_INDEX) VALUES ( {index+1}, {platform_index} )"
                #print( sql )
                cur.execute( sql )
                
                sql = f"INSERT INTO PLATFORM(RADAR_INDEX, PLATFORM_INDEX, PLATFORM) VALUES ( {index+1}, {platform_index}, '{platform_name[i]}' )"
                #print( sql )
                cur.execute( sql )
                
    # 사용자 추가
    user_index = index + 1
    value = 'Z001A'
    sql = f"INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DATE_LAST_REVIEWED, TIME_INACTIVATED) VALUES ( {user_index+1}, '{value}', '레이더_{user_index+1}', 100, 'ACTIVE', 1, 1, '', '', 200 )"
    #print( sql );
    cur.execute( sql )
      
    conn.commit()

    print( '===> 정상적으로 레이더 {}개를 변환했으며, 사용자 레이더 {}개를 추가했습니다.'.format( len(elnot_df), user_index-index ) )
        
except Exception as exception:
    print( exception )
    print( '에러 발생함' )

===> 정상적으로 레이더 279개를 변환했으며, 사용자 레이더 1개를 추가했습니다.


# 레이더모드(라이프싸이클, 레이더모드 정보) 테이블 생성

In [9]:
elnot_df = n_df_1['EmitterNotation'].unique()
elnot_df = np.sort( elnot_df )

radarmode_index = 0

rfSeq = 0
rfIndex = 0

priSeq = 0
priIndex = 0

try:
    for index, value in enumerate(elnot_df):
        rows = n_df_1[ n_df_1['EmitterNotation'] == value ]
        radarmode_name = rows.loc[:,['RadarMode Name']]
        for idx, row in rows.iterrows():
            radarmode_index = radarmode_index + 1
            
            # 레이더모드-사이클 정보 저장
            sql = f"INSERT INTO RADARMODE_LIFECYCLE(RADAR_INDEX, RADARMODE_INDEX, RADARMODE_NAME ) VALUES ({index+1}, {radarmode_index}, '{row['RadarMode Name']}' )"
            #print( sql )
            cur.execute( sql )
                        
            # 레이더모드 정보 저장
            sql = f"INSERT INTO RADARMODE VALUES ({radarmode_index}, '{nowStr}', '', '', '', '', {int(row['Pulse Type'])}, 0.0, 0.0, {int(row['RF Type'])}, {row['FreqLow(MHz)']}, {row['FreqHigh(MHz)']}, {row['CenterFreq(MHz)']-5}, {row['CenterFreq(MHz)']+5}, \
0, {row['SwitchLevel']}, {row['PatternType']}, {row['PatternPeriodLow(usec)']}, {row['PatternPeriodHigh(usec)']}, 0.0, {int(row['PRI Type'])}, {row['PRILow(usec)']}, {row['PRIHigh(usec)']}, {row['CenterPRI(usec)']-5}, {row['CenterPRI(usec)']+5}, 0, {row['Stagger/Switch Level']}, {row['PRI Pattern Type']}, {row['PRI PatternPeriodLow(usec)']}, {row['PRI PatternPeriodHigh(usec)']}, \
0, 0, {row['PW Low(usec)']}, {row['PW High(usec)']}, {int(row['AS Type'])}, {int(row['AS Low(usec)'])}, {int(row['AS High(usec)'])}, 0, 0.0, 0.0, 'ZZ', 100, 0, 0, 1 )"            
            #print( sql )
            cur.execute( sql )
            
            # 주파수 레벨값 저장
            bRFSequenceName = True
            for i in range( int( row['SwitchLevel'] ) ):                
                rfIndex = rfIndex + 1
                
                filterLowStr = f"Dwell Freq{i+1} Low(MHz)"
                filterHighStr = f"Dwell Freq{i+1} High(MHz)"
                sql = f"INSERT INTO RADAR_RF_VALUES (RADARMODE_INDEX, RF_MIN, RF_MAX ) VALUES ( {radarmode_index}, {row[filterLowStr]}, {row[filterHighStr]} )"
                #print( filterLowStr, filterHighStr, sql )
                cur.execute( sql )
                
                if bRFSequenceName == True:
                    bRFSequenceName = False
                    rfSeq = rfSeq + 1
                    sql = f"INSERT INTO RADAR_RF_SEQUENCE_NAME (RADARMODE_INDEX, RF_SEQ_ID, RF_SEQ_NAME ) VALUES ( {radarmode_index}, {rfSeq}, 'RF 시퀀스#{rfSeq}' )"
                    print( sql )
                    #cur.execute( sql )
                
                sql = f"INSERT INTO RADAR_RF_SEQUENCE (RADARMODE_INDEX, RF_SEQ_ID, RF_INDEX) VALUES ( {radarmode_index}, {rfSeq}, {rfIndex} )"
                print( sql )
                #cur.execute( sql )            
            
            # PRI 레벨값 저장
            bPriSequenceName = True
            for i in range( int( row['Stagger/Switch Level'] ) ):                
                priIndex = priIndex + 1
                
                filterLowStr = f"SD Value{i+1} Low(MHz)"
                filterHighStr = f"SD Value{i+1} High(MHz)"
                sql = f"INSERT INTO RADAR_PRI_VALUES (RADARMODE_INDEX, PRI_MIN, PRI_MAX, DWELL_DURATION_MIN, DWELL_DURATION_MAX, PULSES_PER_DWELL, TRANSITION_MIN, TRANSITION_MAX) VALUES ( {radarmode_index}, {row[filterLowStr]}, {row[filterHighStr]}, 0.0, 0.0, 100, 0.0, 0.0 )"
                #print( filterLowStr, filterHighStr, sql )
                cur.execute( sql )

                if bPriSequenceName == True:
                    bPriSequenceName = False
                    priSeq = priSeq + 1
                    sql = f"INSERT INTO RADAR_PRI_SEQUENCE_NAME (RADARMODE_INDEX, PRI_SEQ_ID, PRI_SEQ_NAME ) VALUES ( {radarmode_index}, {priSeq}, 'PRI 시퀀스#{priSeq}' )"
                    #print( sql )
                    cur.execute( sql )
                
                sql = f"INSERT INTO RADAR_PRI_SEQUENCE (RADARMODE_INDEX, PRI_SEQ_ID, PRI_INDEX) VALUES ( {radarmode_index}, {priSeq}, {priIndex} )"
                #print( sql )
                cur.execute( sql )
            
    # 사용자 레이러 모드 추기
    user_index = index + 1
    user_radarmode_index = radarmode_index + 1
    
    # 레이더모드-사이클 정보 저장        
    sql = f"INSERT INTO RADARMODE_LIFECYCLE(RADAR_INDEX, RADARMODE_INDEX, RADARMODE_NAME ) VALUES ({user_index+1}, {user_radarmode_index}, '사용자 레이더모드' )"
    #print( sql )
    cur.execute( sql )
    
    sql = f"INSERT INTO RADARMODE VALUES ({user_radarmode_index}, '{nowStr}', '', '', '', '', 1, 0.0, 0.0, 0, 500, 18000, 7500, 7500, 0, 0, 0, 0, 0, 0.0, \
0, 100, 100, 95, 105, 0, 0, 0, 0, 0, \
0, 0, 0, 200, 0, 0, 0, 0, 0.0, 0.0, 'ZZ', 100, 0, 0, 1 )"            
    print( sql )
    cur.execute( sql )
    
    # RF 레벨값 저장
    bRFSequenceName = True
    for i in range(16):
        rfIndex = rfIndex + 1
        
        iFreqLow = 6000 + ( i * 10 )
        iFreqHigh = 6500 + ( i * 10 )
        sql = f"INSERT INTO RADAR_RF_VALUES (RADARMODE_INDEX, RF_MIN, RF_MAX ) VALUES ( {user_radarmode_index}, {iFreqLow}, {iFreqHigh} )"    
        #print( sql )
        cur.execute( sql )
        
        if bRFSequenceName == True:
            bRFSequenceName = False
            rfSeq = rfSeq + 1
            sql = f"INSERT INTO RADAR_RF_SEQUENCE_NAME (RADARMODE_INDEX, RF_SEQ_ID, RF_SEQ_NAME ) VALUES ( {user_radarmode_index}, {rfSeq}, 'RF 시퀀스#{rfSeq}' )"
            #print( sql )
            cur.execute( sql )
            
        sql = f"INSERT INTO RADAR_RF_SEQUENCE (RADARMODE_INDEX, RF_SEQ_ID, RF_INDEX) VALUES ( {radarmode_index}, {rfSeq}, {rfIndex} )"
        #print( sql )
        cur.execute( sql ) 
        
    # PRI 레벨값 저장
    bPriSequenceName = True
    for i in range(16):              
        priIndex = priIndex + 1

        # PRI 레벨값 저장
        iPRILow = 200 + ( i * 10 )
        iPRIHigh = iPRILow + 10
        sql = f"INSERT INTO RADAR_PRI_VALUES (RADARMODE_INDEX, PRI_MIN, PRI_MAX, DWELL_DURATION_MIN, DWELL_DURATION_MAX, PULSES_PER_DWELL, TRANSITION_MIN, TRANSITION_MAX) VALUES ( {radarmode_index}, {row[filterLowStr]}, {row[filterHighStr]}, 0.0, 0.0, 100, 0.0, 0.0 )"
        #print( filterLowStr, filterHighStr, sql )
        cur.execute( sql )

        if bPriSequenceName == True:
            bPriSequenceName = False
            priSeq = priSeq + 1
            sql = f"INSERT INTO RADAR_PRI_SEQUENCE_NAME (RADARMODE_INDEX, PRI_SEQ_ID, PRI_SEQ_NAME ) VALUES ( {radarmode_index}, {priSeq}, 'PRI 시퀀스#{priSeq}' )"
            #print( sql )
            cur.execute( sql )

        sql = f"INSERT INTO RADAR_PRI_SEQUENCE (RADARMODE_INDEX, PRI_SEQ_ID, PRI_INDEX) VALUES ( {radarmode_index}, {priSeq}, {priIndex} )"
        #print( sql )
        cur.execute( sql )        
        
    conn.commit()            
    print( '===> 정상적으로 레이더 모드 {}개를 변환했고, 사용자 레이더 모드 {}개를 변환했습니다.'.format( radarmode_index, user_radarmode_index-radarmode_index ) )

except Exception as exception:
    print( exception )
    print( '에러 발생 !' )   
    

INSERT INTO RADARMODE VALUES (2001, '2023-01-06 11:01:52', '', '', '', '', 1, 0.0, 0.0, 0, 500, 18000, 7500, 7500, 0, 0, 0, 0, 0, 0.0, 0, 100, 100, 95, 105, 0, 0, 0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0, 0.0, 0.0, 'ZZ', 100, 0, 0, 1 )
===> 정상적으로 레이더 모드 2000개를 변환했고, 사용자 레이더 모드 1개를 변환했습니다.


# 위협/장비 테이블 삭제

In [10]:
now = datetime.now()
nowStr = now.strftime( '%Y-%m-%d %H:%M:%S' )

print( f"현재 시간은 {nowStr} 입니다.\n\n" )

try:
    cur.execute( 'delete from DEVICE;' )
    cur.execute( 'delete from THREAT;')
    
    print( "===> 테이블을 정상적으로 삭제 했습니다." )
    
    
    print( "===> 자동 증가를 초기화 했습니다." )

    conn.commit()
    
except Exception as exception:
    print( exception )
    print( '에러 발생함' )   


현재 시간은 2023-01-06 11:02:37 입니다.


===> 테이블을 정상적으로 삭제 했습니다.
===> 자동 증가를 초기화 했습니다.


# 위협/장비 테이블 생성

In [11]:
def RandomFloat32( start_value, end_value ):
    return random.random() * ( end_value - start_value ) + start_value;

try:    
    for index, value in enumerate(elnot_df):
        # 위경도 랜던 모의
        latitude = RandomFloat32( 38, 39 );
        longitude = RandomFloat32( 127, 129 );
        #print( latitude, longitude )
        
        # 위협 정보 저장
        sql = f"INSERT INTO THREAT (THREAT_INDEX, THREAT_NAME, LATITUDE, LONGITUDE, SITE_NAME, PIN, PLACE_NAME_KOR, FRIEND_OR_FOE, PRIORITY, CATEGORY, PLATFORM_TYPE, SYMBOL_CODE ) VALUES ( {index+1}, '위협명 #{index+1}', '{latitude}', '{longitude}', '지명#{index+1}', 1, '한국지명', '적', 100, 1, 1, '2525B 심벌' );";
        #print( sql )
        cur.execute( sql )
             
        # 장비 정보 저정
        device_index = 1
        sql = f"INSERT INTO DEVICE (THREAT_INDEX, DEVICE_INDEX, ELNOT, DEVICE_NAME, IDENTIFICATION_RANGE, LATITUDE, LONGITUDE, ALTITUDE ) VALUES ( {index+1}, {device_index}, '{value}', '장비명 #{index+1}', 10.0, '{latitude}', '{longitude}', 10.0 );";
        #print( sql )      
        cur.execute( sql )
      
    conn.commit()

    print( '===> 정상적으로 위협/장비 {}/{}개를 생성했습니다.'.format( len(elnot_df), len(elnot_df) ) )
        
except Exception as exception:
    print( exception )
    print( '에러 발생함' )

===> 정상적으로 위협/장비 279/279개를 생성했습니다.


# CEDEOB DB 파일 복사

In [12]:
! copySqlite.bat


E:\APECS\Dev\PocketSONATA\dev_\Document\IPL2CED>echo off 
        1개 파일이 복사되었습니다.


오류: 입력 리디렉션이 지원되지 않습니다. 프로세스를 즉시 끝냅니다.


In [12]:
conn.close()

print( "DB 를 종료합니다 !!!" )

DB 를 종료합니다 !!!
